## Maximum Temperature Processing Script

Start by importing modules and setting up ESGF search

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr

%matplotlib inline

/home/nick/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/nick/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
#!/usr/bin/env python
from __future__ import print_function
import requests
import xml.etree.ElementTree as ET

# Author: Unknown
# I got the original version from a word document published by ESGF
# https://docs.google.com/document/d/1pxz1Kd3JHfFp8vR2JCVBfApbsHmbUQQstifhGNdc6U0/edit?usp=sharing

# API AT: https://github.com/ESGF/esgf.github.io/wiki/ESGF_Search_REST_API#results-pagination

def esgf_search(server="https://esgf-node.llnl.gov/esg-search/search",
                files_type="OPENDAP", local_node=True, project="CMIP6",
                verbose=False, format="application%2Fsolr%2Bjson",
                use_csrf=False, **search):
    client = requests.session()
    payload = search
    payload["project"] = project
    payload["type"]= "File"
    if local_node:
        payload["distrib"] = "false"
    if use_csrf:
        client.get(server)
        if 'csrftoken' in client.cookies:
            # Django 1.6 and up
            csrftoken = client.cookies['csrftoken']
        else:
            # older versions
            csrftoken = client.cookies['csrf']
        payload["csrfmiddlewaretoken"] = csrftoken

    payload["format"] = format

    offset = 0
    numFound = 10000
    all_files = []
    files_type = files_type.upper()
    while offset < numFound:
        payload["offset"] = offset
        url_keys = []
        for k in payload:
            url_keys += ["{}={}".format(k, payload[k])]

        url = "{}/?{}".format(server, "&".join(url_keys))
        print(url)
        r = client.get(url)
        r.raise_for_status()
        resp = r.json()["response"]
        numFound = int(resp["numFound"])
        resp = resp["docs"]
        offset += len(resp)
        for d in resp:
            if verbose:
                for k in d:
                    print("{}: {}".format(k,d[k]))
            url = d["url"]
            for f in d["url"]:
                sp = f.split("|")
                if sp[-1] == files_type:
                    all_files.append(sp[0].split(".html")[0])
    return sorted(all_files)

In [ ]:
##Use this module to search for files:
model = 'ACCESS-CM2'
activity = 'CMIP'#'ScenarioMIP' #CMIP
scenario = 'abrupt-4xCO2'#ssp245' #piControl #abrupt4xCO2
variable = 'tas'

result = esgf_search(activity_id=activity, table_id='day', variable_id=variable,
                                 experiment_id = scenario, source_id = model)
files_to_open = result[:]
print(result)

https://esgf-node.llnl.gov/esg-search/search/?activity_id=CMIP&variable_id=tas&format=application%2Fsolr%2Bjson&project=CMIP6&distrib=false&table_id=day&experiment_id=abrupt-4xCO2&offset=0&source_id=ACCESS-CM2&type=File
[u'https://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/abrupt-4xCO2/r1i1p1f1/day/tas/gn/v20191108/tas_day_ACCESS-CM2_abrupt-4xCO2_r1i1p1f1_gn_09500101-09991231.nc', u'https://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/abrupt-4xCO2/r1i1p1f1/day/tas/gn/v20191108/tas_day_ACCESS-CM2_abrupt-4xCO2_r1i1p1f1_gn_10000101-10491231.nc', u'https://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/abrupt-4xCO2/r1i1p1f1/day/tas/gn/v20191108/tas_day_ACCESS-CM2_abrupt-4xCO2_r1i1p1f1_gn_10500101-10991231.nc']


In [ ]:
##Use this module to search for land files:
model = 'ACCESS-CM2'
activity = 'CMIP' #CMIP
scenario = 'piControl' #piControl #abrupt4xCO2
variable = 'sftlf'

result = esgf_search(activity_id=activity, table_id='fx', variable_id=variable,
                                 experiment_id = scenario, source_id = model)
files_to_open = result[:]
print(result)

#Print out land file resolution
ds = xr.open_dataset(files_to_open[0])
print(np.shape(ds.sftlf[:, :]))

https://esgf-node.llnl.gov/esg-search/search/?activity_id=CMIP&variable_id=sftlf&format=application%2Fsolr%2Bjson&project=CMIP6&distrib=false&table_id=fx&experiment_id=piControl&offset=0&source_id=ACCESS-CM2&type=File
[u'http://esgf-data04.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/piControl/r1i1p1f1/fx/sftlf/gn/v20191112/sftlf_fx_ACCESS-CM2_piControl_r1i1p1f1_gn.nc', u'http://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/piControl/r1i1p1f1/fx/sftlf/gn/v20191112/sftlf_fx_ACCESS-CM2_piControl_r1i1p1f1_gn.nc']
(144, 192)


# First do tas

## SSP2.45

In [ ]:
#models which claim to have run these experiments:
models = ["ACCESS-CM2", "ACCESS-ESM1-5", "AWI-CM-1-1-MR", "CAMS-CSM1-0", "CMCC-CM2-SR5", "CMCC-ESM2",
          "CNRM-CM6-1","CNRM-CM6-1-HR", "CNRM-ESM2-1", "EC-Earth3", "EC-Earth3-CC",
          "EC-Earth3-Veg", "EC-Earth3-Veg-LR", "GFDL-CM4",  "GFDL-ESM4", "HadGEM3-GC31-LL",
         "IITM-ESM", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR",
          "NorESM2-LM", "NorESM2-MM", "TaiESM1",
          "UKESM1-0-LL", "UKESM1-1-LL"]
m = len(models)

lim = [2, 2, 86, 1, 4, 4, 1, 2, 1, 86, 86, 86, 86, 5, 5,  2, 17, 2, 2, 1, 9, 9, 9, 2, 2 ]

issues = ["CAMS-CSM1-0","CNRM-CM6-1","CNRM-ESM2-1","GFDL-CM4"]

In [ ]:
#Use these to open land files
land_models = ["ACCESS-CM2", "ACCESS-ESM1-5", "AWI-CM-1-1-MR", "CAMS-CSM1-0", "CMCC-CM2-SR5", "CMCC-ESM2",
          "CNRM-CM6-1","CNRM-CM6-1-HR", "CNRM-ESM2-1", "EC-Earth3", "EC-Earth3-CC",
          "EC-Earth3-Veg", "EC-Earth3-Veg-LR", "GFDL-CM4",  "GFDL-ESM4", "HadGEM3-GC31-LL",
         "IITM-ESM", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR",
          "NorESM2-LM", "NorESM2-MM", "TaiESM1",
          "UKESM1-0-LL", "UKESM1-1-LL"]

In [ ]:
#Loop through models and calculate global-mean temperature, land temperatures and maximum daily-mean temperatures

for i in range(1):
    print("doing", models[i])
    result = esgf_search(activity_id='ScenarioMIP', table_id='day', variable_id='tas',
                                 experiment_id = 'ssp245', source_id = models[i])

    if lim[i] == 1:
        files_to_open = result[:]
        ds = xr.open_dataset(files_to_open[0])
    elif i == 13 or i == 19:
        files_to_open = result[:lim[i]]
        ds = xr.open_mfdataset(files_to_open, chunks={'time': '100MB'}, decode_times=False)
    else:
        files_to_open = result[:lim[i]]
        ds = xr.open_mfdataset(files_to_open)

    result2 = esgf_search(activity_id='CMIP', table_id='fx', variable_id='sftlf',
                                 experiment_id = 'piControl', source_id = land_models[i])

    ds2 = xr.open_dataset(result2[0])
    land_mask = ds2.sftlf[:, :]
    mask = np.ma.masked_where(land_mask > 50., land_mask)
    mask = np.ma.getmask(mask)

    print("loaded data, get annual maxima")

    lat = ds.lat[:]
    lon = ds.lon[:]

    rad_lat = lat * np.pi / 180.
    rad_lon = lon * np.pi / 180.
    cos_lat = np.cos(rad_lat)

    #average between 45N and 45N
    l1 = np.where(lat > -45.)
    l1 = np.min(l1)
    l2 = np.where(lat > 45.)
    l2 = np.min(l2)

    weighting = mask[l1:l2] * np.expand_dims(cos_lat[l1:l2], axis = 1)
    global_mean = np.sum(weighting)
    weighting /= global_mean

    if i == 15:
        l = 85
    else:
        l = 86

    mean_land_temps = np.zeros(l) #array to store data
    mean_temps = np.zeros(l) #array to store data
    max_temps = np.zeros(l) #array to store data

    for j in range( 1 ):
        if j % 10 == 0:
            print(j)
        land_temp = ds.tas[j * 365:(j + 1) * 365, l1:l2] * np.expand_dims(weighting[:], axis = 0)
        print(land_temp, np.sum(land_temp))
        mean_land_temps[j] = np.sum(land_temp) / 365.
    """
        yearly_mean = np.mean(ds.tas[j * 365:(j + 1) * 365], axis = 0)
        mean_temps[j] = np.trapz(np.trapz(yearly_mean * np.expand_dims(cos_lat, axis = 1), rad_lon, axis = 1), rad_lat, axis = 0) / 4. / np.pi

        max_temps[j] = np.max(ds.tas[j * 365:(j + 1) * 365, :])

    #Save's
    np.save("ssp_data/" + models[i] + "_max_daily_temp", max_temps)
    np.save("ssp_data/" + models[i] + "_mean_temp", mean_temps)
    np.save("ssp_data/" + models[i] + "_mean_land_temp", mean_land_temps)
    """

doing ACCESS-CM2
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&variable_id=tas&format=application%2Fsolr%2Bjson&project=CMIP6&distrib=false&table_id=day&experiment_id=ssp245&offset=0&source_id=ACCESS-CM2&type=File
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&variable_id=tas&format=application%2Fsolr%2Bjson&project=CMIP6&distrib=false&table_id=day&experiment_id=ssp245&offset=10&source_id=ACCESS-CM2&type=File
https://esgf-node.llnl.gov/esg-search/search/?activity_id=CMIP&variable_id=sftlf&format=application%2Fsolr%2Bjson&project=CMIP6&distrib=false&table_id=fx&experiment_id=piControl&offset=0&source_id=ACCESS-CM2&type=File
loaded data, get annual maxima
0
<xarray.DataArray 'tas' (time: 365, lat: 72, lon: 192)>
dask.array<shape=(365, 72, 192), dtype=float64, chunksize=(365, 72, 192)>
Coordinates:
  * lat      (lat) float64 -44.38 -43.12 -41.88 -40.62 ... 41.88 43.12 44.38
  * lon      (lon) float64 0.9375 2.812 4.688 6.562 ... 353.4 355.3 3

In [ ]:
print(np.sum(land_temp))

<xarray.DataArray 'tas' ()>
dask.array<shape=(), dtype=float64, chunksize=()>
Coordinates:
    height   float64 2.0


## Abrupt 4XCO2

In [ ]:
#models which claim to have run these experiments (might be different from above)
models = ["ACCESS-CM2", "ACCESS-ESM1-5", "AWI-CM-1-1-MR", "CAMS-CSM1-0", "CMCC-CM2-SR5", "CMCC-ESM2",
          "CNRM-CM6-1","CNRM-CM6-1-HR", "CNRM-ESM2-1", "EC-Earth3", "EC-Earth3-CC",
          "EC-Earth3-Veg", "EC-Earth3-Veg-LR", "GFDL-CM4",  "GFDL-ESM4", "HadGEM3-GC31-LL",
         "IITM-ESM", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR",
          "NorESM2-LM", "NorESM2-MM", "TaiESM1",
          "UKESM1-0-LL", "UKESM1-1-LL"]
m = len(models)

lim = [3, 2, 86, 1, 4, 4, 1, 2, 1, 86, 86, 86, 86, 5, 5,  2, 17, 2, 2, 1, 9, 9, 9, 2, 2 ]

issues = ["CAMS-CSM1-0","CNRM-CM6-1","CNRM-ESM2-1","GFDL-CM4"]

In [ ]:
#Loop through models and calculate global-mean temperature, land temperatures and maximum daily-mean temperatures

for i in range(1):
    print("doing", models[i])
    result = esgf_search(activity_id='CMIP', table_id='day', variable_id='tas',
                                 experiment_id = 'abrupt-4xCO2', source_id = models[i])

    if lim[i] == 1:
        files_to_open = result[:]
        ds = xr.open_dataset(files_to_open[0])
    else:
        files_to_open = result[:lim[i]]
        ds = xr.open_mfdataset(files_to_open, chunks={'time': '100MB'}, decode_times=False)

    result2 = esgf_search(activity_id='CMIP', table_id='fx', variable_id='sftlf',
                                 experiment_id = 'piControl', source_id = land_models[i])

    ds2 = xr.open_dataset(result2[0])
    land_mask = ds2.sftlf[:, :]
    mask = np.ma.masked_where(land_mask > 50., land_mask)
    mask = np.ma.getmask(mask)

    print("loaded data, get annual maxima")

    lat = ds.lat[:]
    lon = ds.lon[:]

    rad_lat = lat * np.pi / 180.
    rad_lon = lon * np.pi / 180.
    cos_lat = np.cos(rad_lat)

    #average between 45N and 45N
    l1 = np.where(lat > -45.)
    l1 = np.min(l1)
    l2 = np.where(lat > 45.)
    l2 = np.min(l2)

    weighting = mask[l1:l2] * np.expand_dims(cos_lat[l1:l2], axis = 1)
    global_mean = np.sum(weighting)
    weighting /= global_mean

    l = 140

    mean_land_temps = np.zeros(l) #array to store data
    mean_temps = np.zeros(l) #array to store data
    max_temps = np.zeros(l) #array to store data

    for j in range( l ):
        if j % 10 == 0:
            print(j)
        land_temp = ds.tas[j * 365:(j + 1) * 365, l1:l2] * np.expand_dims(weighting[:], axis = 0)
        mean_land_temps[j] = np.float(np.sum(land_temp) / 365.)

        yearly_mean = np.mean(ds.tas[j * 365:(j + 1) * 365], axis = 0)
        mean_temps[j] = np.trapz(np.trapz(yearly_mean * np.expand_dims(cos_lat, axis = 1), rad_lon, axis = 1), rad_lat, axis = 0) / 4. / np.pi

        max_temps[j] = np.max(ds.tas[j * 365:(j + 1) * 365, :])

    #Save's
    np.save("abrupt4x_data/" + models[i] + "_max_daily_temp", max_temps)
    np.save("abrupt4x_data/" + models[i] + "_mean_temp", mean_temps)
    np.save("abrupt4x_data/" + models[i] + "_mean_land_temp", mean_land_temps)


doing ACCESS-CM2
https://esgf-node.llnl.gov/esg-search/search/?activity_id=CMIP&variable_id=tas&format=application%2Fsolr%2Bjson&project=CMIP6&distrib=false&table_id=day&experiment_id=abrupt-4xCO2&offset=0&source_id=ACCESS-CM2&type=File
https://esgf-node.llnl.gov/esg-search/search/?activity_id=CMIP&variable_id=sftlf&format=application%2Fsolr%2Bjson&project=CMIP6&distrib=false&table_id=fx&experiment_id=piControl&offset=0&source_id=ACCESS-CM2&type=File
loaded data, get annual maxima
0


ValueError: setting an array element with a sequence.

## Pi Control

want these to take difference with abrupt-4x runs

In [ ]:
#models which claim to have run these experiments (might be different from above)
models = ["ACCESS-CM2", "ACCESS-ESM1-5", "AWI-CM-1-1-MR", "CAMS-CSM1-0", "CMCC-CM2-SR5", "CMCC-ESM2",
          "CNRM-CM6-1","CNRM-CM6-1-HR", "CNRM-ESM2-1", "EC-Earth3", "EC-Earth3-CC",
          "EC-Earth3-Veg", "EC-Earth3-Veg-LR", "GFDL-CM4",  "GFDL-ESM4", "HadGEM3-GC31-LL",
         "IITM-ESM", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR",
          "NorESM2-LM", "NorESM2-MM", "TaiESM1",
          "UKESM1-0-LL", "UKESM1-1-LL"]
m = len(models)

lim = [3, 2, 86, 1, 4, 4, 1, 2, 1, 86, 86, 86, 86, 5, 5,  2, 17, 2, 2, 1, 9, 9, 9, 2, 2 ]

issues = ["CAMS-CSM1-0","CNRM-CM6-1","CNRM-ESM2-1","GFDL-CM4"]

In [ ]:
#Loop through models and calculate global-mean temperature, land temperatures and maximum daily-mean temperatures

for i in range(1):
    print("doing", models[i])
    result = esgf_search(activity_id='CMIP', table_id='day', variable_id='tas',
                                 experiment_id = 'piControl', source_id = models[i])

    if lim[i] == 1:
        files_to_open = result[:]
        ds = xr.open_dataset(files_to_open[0])
    else:
        files_to_open = result[:lim[i]]
        ds = xr.open_mfdataset(files_to_open, chunks={'time': '100MB'}, decode_times=False)


    result2 = esgf_search(activity_id='CMIP', table_id='fx', variable_id='sftlf',
                                 experiment_id = 'piControl', source_id = land_models[i])

    ds2 = xr.open_dataset(result2[0])
    land_mask = ds2.sftlf[:, :]
    mask = np.ma.masked_where(land_mask > 50., land_mask)
    mask = np.ma.getmask(mask)

    print("loaded data, get annual maxima")

    lat = ds.lat[:]
    lon = ds.lon[:]

    rad_lat = lat * np.pi / 180.
    rad_lon = lon * np.pi / 180.
    cos_lat = np.cos(rad_lat)

    #average between 45N and 45N
    l1 = np.where(lat > -45.)
    l1 = np.min(l1)
    l2 = np.where(lat > 45.)
    l2 = np.min(l2)

    weighting = mask[l1:l2] * np.expand_dims(cos_lat[l1:l2], axis = 1)
    global_mean = np.sum(weighting)
    weighting /= global_mean

    if i == 15:
        l = 85
    else:
        l = 86

    mean_land_temps = np.zeros(l) #array to store data
    mean_temps = np.zeros(l) #array to store data
    max_temps = np.zeros(l) #array to store data

    for j in range( l ):
        if j % 10 == 0:
            print(j)
        land_temp = ds.tas[j * 365:(j + 1) * 365, l1:l2] * np.expand_dims(weighting[:], axis = 0)
        mean_land_temps[j] = np.sum(land_temp) / 365.

        yearly_mean = np.mean(ds.tas[j * 365:(j + 1) * 365], axis = 0)
        mean_temps[j] = np.trapz(np.trapz(yearly_mean * np.expand_dims(cos_lat, axis = 1), rad_lon, axis = 1), rad_lat, axis = 0) / 4. / np.pi

        max_temps[j] = np.max(ds.tas[j * 365:(j + 1) * 365, :])

    #Save's
    np.save("picontrol_data/" + models[i] + "_max_daily_temp", max_temps)
    np.save("picontrol_data/" + models[i] + "_mean_temp", mean_temps)
    np.save("picontrol_data/" + models[i] + "_mean_land_temp", mean_land_temps)
